In [9]:
import requests
from bs4 import BeautifulSoup
import json
def get_kaist_pure_data():
    url = 'https://pure.kaist.ac.kr/en/persons/'
    headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'accept-language': 'ko,en-US;q=0.9,en;q=0.8',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36'
    }

    response = requests.get(url, headers=headers)
    print(response.text)
    soup = BeautifulSoup(response.text, 'html.parser')

    print(soup)
    researchers = []
    for item in soup.select('.grid-result-item'):
        researcher = {}
        
        # Basic info
        name_elem = item.select_one('.rendering_person .title span')
        if name_elem:
            researcher['name'] = name_elem.text.strip()
            
        profile_elem = item.select_one('.rendering_person .title a')
        if profile_elem:
            researcher['profile_url'] = profile_elem['href']
        
        # Profile image
        img_elem = item.select_one('img')
        if img_elem:
            researcher['image_url'] = img_elem['src']
            researcher['image_alt'] = img_elem['alt']
        
        # Email (if available)
        email_elem = item.select_one('.email a')
        if email_elem:
            # Handle encrypted email format
            email_parts = []
            for part in email_elem.stripped_strings:
                email_parts.append(part.strip())
            researcher['email'] = ''.join(email_parts)
            
        # Organization
        org_elem = item.select_one('.organisations a span')
        if org_elem:
            researcher['organization'] = org_elem.text.strip()
            
        # Person type
        type_elem = item.select_one('.type span.family')
        if type_elem:
            researcher['type'] = type_elem.text.replace('Person: ', '').strip()
            
        # Research output trend
        trend_data = {}
        trend_graph = item.select_one('.stacked-trend-graph')
        if trend_graph:
            for bar in trend_graph.select('.trend-bar-stacked'):
                info = bar['data-graph-info']
                year = info.split(':')[0].split()[-1]
                count = int(info.split(':')[1].strip())
                trend_data[year] = count
                
        researcher['research_output'] = trend_data
        
        researchers.append(researcher)
        
    return researchers

# Execute and print results
researchers = get_kaist_pure_data()
print(json.dumps(researchers, indent=2))

<!DOCTYPE html><html lang="en-US"><head><title>Just a moment...</title><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=Edge"><meta name="robots" content="noindex,nofollow"><meta name="viewport" content="width=device-width,initial-scale=1"><style>*{box-sizing:border-box;margin:0;padding:0}html{line-height:1.15;-webkit-text-size-adjust:100%;color:#313131;font-family:system-ui,-apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Helvetica Neue,Arial,Noto Sans,sans-serif,Apple Color Emoji,Segoe UI Emoji,Segoe UI Symbol,Noto Color Emoji}body{display:flex;flex-direction:column;height:100vh;min-height:100vh}.main-content{margin:8rem auto;max-width:60rem;padding-left:1.5rem}@media (width <= 720px){.main-content{margin-top:4rem}}.h2{font-size:1.5rem;font-weight:500;line-height:2.25rem}@media (width <= 720px){.h2{font-size:1.25rem;line-height:1.5rem}}#challenge-error-text{background-image:url();background-repeat:no-repeat;background-s

In [ ]:
## pure는 http로 보낼때 복호화함.